In [33]:
import plotly.figure_factory as ff
import pandas as pd
import numpy as np
from dash import Dash, html, dcc, Input, Output, callback, dash_table, State
from datetime import datetime as dt
from datetime import timedelta
import dash_bootstrap_components as dbc
import dash_html_components as html
import dash_core_components as dcc
import plotly.graph_objects as go
import random
import plotly.express as px
from plotly.subplots import make_subplots
from dash import html
import os
import dash
import datetime
import subprocess
import webbrowser
from dash.dependencies import Input, Output
from flask import Flask, render_template, jsonify, request

In [7]:
#CONTENT STYLE

def display_page(pathname):
    content_style = {
        'minHeight': '560px',
        'borderRadius': '5px',
        'border': '1px solid red',
        'margin': '10px',
        'padding': '10px',
        'backgroundColor': '#ffffff',
        'overflowX': 'auto',  # Enable horizontal scrolling
        'overflowY': 'auto'   # Enable vertical scrolling
    }

# Function to generate the Gantt chart
def generate_gantt_chart(data):
    # Convert date columns to datetime
    data['EXPECTED START'] = pd.to_datetime(data['EXPECTED START'], format='%d/%m/%Y', errors='coerce')
    data['DUE.DATE'] = pd.to_datetime(data['DUE.DATE'], format='%d/%m/%Y', errors='coerce')

    # Filter out rows without dates
    data = data.dropna(subset=['EXPECTED START', 'DUE.DATE'], how='all')

    # Debugging: Print the data to ensure it's not empty
    print("Data for Gantt Chart:")
    print(data)

    # Create Gantt chart
    fig = px.timeline(
        data,
        x_start="EXPECTED START",
        x_end="DUE.DATE",
        y="P_N",
        color_discrete_sequence=['#FFBF00'],
        title="Gantt Chart"
    )

    # Update layout
    fig.update_layout(
        xaxis_title="Date",
        yaxis_title="Tasks",
        yaxis=dict(autorange="reversed"),  # Reverse the y-axis to have the earliest tasks at the top
        showlegend=False
    )

    return fig

# Functions ____________________________________________________________

#1____
def timedelta_to_str(td):
    return str(td)
#2____
# Function to standardize date format
def standardize_date(date_str):
    for fmt in ("%Y-%m-%d %H:%M:%S", "%Y-%m-%d", "%d-%m-%Y", "%d/%m/%Y"):
        try:
            return pd.to_datetime(date_str, format=fmt).strftime('%d/%m/%Y')
        except ValueError:
            pass
    try:
        return pd.to_datetime(float(date_str), unit='d', origin='1899-12-30').strftime('%d/%m/%Y')
    except (ValueError, TypeError):
        return date_str  # Return the original string if no format matches

In [5]:
database = pd.read_excel('total_data_merged.xlsm', header = 0, sheet_name = 'data323_comma') # Import data from new database
database = database.dropna(how='all')
database.dropna(axis=1, how='all', inplace=True)

desired_columns = [
    'EXTRACTED TEXT', 'NUMERO ODL', 'Column1', 'CODICE MACCHINA', 'MATRICOLA MACCHINA', 'REVISIONE', 
    'GAS REFRIGERANTE', 'FLUIDO', 'NR CIRCUITI', 'TEST.ID2', 'N.PROVA', 'STAZIONE DI COLLAUDO', 
    'OPERATORE', 'LINEA DEL FLUIDO', 'ALIMENTAZIONE', 'ESITO COLLAUDO', 'Duration', 'P1', 'P2', 
    'P3', 'P4', 'P5', 'P6', 'T1', 'T2', 'T3', 'T4', 'T5', 'T6', 'TC7', 'TC8', 'TC9', 'TC10', 
    'P7', 'P8', 'P9', 'P10', 'PRELE_1', 'VCM_1', 'ITOT_1', 'PACTT_1', 'CFTOT_1', 'FTOT_1', 
    'VCM_2', 'ITOT_2', 'PACTT_2', 'CFTOT_2', 'FTOT_2', 'VCM_3', 'ITOT_3', 'PACTT_3', 'CFTOT_3', 
    'FTOT_3', 'VCM_4', 'ITOT_4', 'PACTT_4', 'CFTOT_4', 'FTOT_4', 'VCM_5', 'ITOT_5', 'PACTT_5', 
    'CFTOT_5', 'FTOT_5', 'EVAP1_POWER', 'EVAP1_TIN_M', 'EVAP1_TOUT_M', 'EVAP2_POWER', 
    'EVAP2_TIN_M', 'EVAP2_TOUT_M', 'EVAP3_POWER', 'EVAP3_TIN_M', 'EVAP3_TOUT_M', 'TC_P7', 
    'TC_P8', 'TC_P9', 'TC_P10', 'SOTT_1', 'SOTT_2', 'SURR_1', 'SURR_2', 'TAC_M', 'MP1_2', 
    'MP3_4', 'MP5_6', 'RESPONSABILE LINEA CHILLER\\CLIMA', 'DIREZIONE TECNICA', 'File name', 
    'Last modification', 'Note'
]
# Filter the desired_columns to include only those that exist in the dataframe
existing_columns = [col for col in desired_columns if col in database.columns]

# Reorder the columns and delete non-listed index
database = database[existing_columns]

# Rename the columns as per the given list
new_column_names = [
    "Date", "Order", "Product family", "Part number", "Serial number", "Revision",
    "Refrigerant", "Fluid", "Circuits", "Test ID", "Test number", "Cell",
    "Operator", "Feeding line", "Electrical Supply", "Test status", "Duration",
    "Pressione olio IN A", "Pressione olio OUT A", "Pressione acqua IN B",
    "Pressione acqua OUT B", "Pressione acqua IN C", "Pressione acqua OUT C",
    "Temper. olio IN A", "Temper. olio OUT A", "Temper. acqua IN B",
    "Temper. acqua OUT B", "Temper. acqua IN C", "Temper. acqua OUT C",
    "Temperatura Liquida 1", "Temperatura Liquida 2", "Temperatura Aspirazione 1",
    "Temperatura Aspirazione 2", "Pressione Condensazione 1",
    "Pressione Condensazione 2", "Pressione Evaporazione 1",
    "Pressione Evaporazione 2",
    "Presa elettrica selezionata 1..5 da pann. Operatore",
    "Tensione concatenata media - Presa 1",
    "Corrente totale - Presa 1",
    "Potenza Attiva Totale - Presa 1",
    "Fattore di potenza totale - Presa 1",
    "Frequenza totale - Presa 1",
    "Tensione concatenata media - Presa 2",
    "Corrente totale - Presa 2",
    "Potenza Attiva Totale - Presa 2",
    "Fattore di potenza totale - Presa 2",
    "Frequenza totale - Presa 2",
    "Tensione concatenata media - Presa 3",
    "Corrente totale - Presa 3",
    "Potenza Attiva Totale - Presa 3",
    "Fattore di potenza totale - Presa 3",
    "Frequenza totale - Presa 3",
    "Tensione concatenata media - Presa 4",
    "Corrente totale - Presa 4",
    "Potenza Attiva Totale - Presa 4",
    "Fattore di potenza totale - Presa 4",
    "Frequenza totale - Presa 4",
    "Tensione concatenata media - Presa 5",
    "Corrente totale - Presa 5",
    "Potenza Attiva Totale - Presa 5",
    "Fattore di potenza totale - Presa 5",
    "Frequenza totale - Presa 5",
    "Resa lato evaporatore 1 (Olio)",
    "Temperatura media ingresso evaporatore 1",
    "Temperatura media uscita evaporatore 1",
    "Resa lato evaporatore 2 (Acqua)",
    "Temperatura media ingresso evaporatore 2",
    "Temperatura media uscita evaporatore 2",
    "Resa lato evaporatore 2 (Glycol)",
    "Temperatura media ingresso evaporatore 3",
    "Temperatura media uscita evaporatore 3",
    "Temperatura condensazione 1 calcolata (da P7)",
    "Temperatura condensazione 2 calcolata (da P8)",
    "Temperatura evaporazione 1 calcolata (da P9)",
    "Temperatura evaporazione 2 calcolata (da P10)",
    "Sottoraffreddamento 1 (TC7-TC_P7)",
    "Sottoraffreddamento 2 (TC8-TC_P8)",
    "Surriscaldamento 1 (TC9-TC_P9)",
    "Surriscaldamento 2 (TC10-TC_P10)",
    "Temperatura aria cabina media",
    "Portata totale evaporatore A",
    "Portata totale evaporatore B",
    "Portata totale evaporatore C",
    "Manager",
    "Director",
    "File name",
    "Last modified",
    "Note"
]

# Rename the columns
database.columns = new_column_names

style_cell={
                    'minWidth': '150px', 'width': '150px', 'maxWidth': '150px',  # Set cell width
                    'textAlign': 'center'  # Centralize text in cells
                }

# Apply the function to the "Date" column
database['Date'] = database['Date'].apply(standardize_date)

# Convert the DataFrame to a dictionary
data = database.to_dict('records')

# Function to convert timedelta to string (if needed)
def timedelta_to_str(td):
    return str(td)

# Format the data for the DataTable
for record in data:
    for key, value in record.items():
        if isinstance(value, datetime.timedelta):
            record[key] = timedelta_to_str(value)
        elif key == 'Date':
            record[key] = standardize_date(value)



In [31]:
#HTML TEMPLATE

html_template = """
<!DOCTYPE html>
<html>
<head>
    <meta charset="UTF-8">
    <title>Laboratory Dashboard</title>
    <link rel="stylesheet" href="https://stackpath.bootstrapcdn.com/bootstrap/4.5.2/css/bootstrap.min.css">
    <script src="https://code.jquery.com/jquery-3.5.1.slim.min.js"></script>
    <script src="https://cdn.jsdelivr.net/npm/@popperjs/core@2.9.2/dist/umd/popper.min.js"></script>
    <script src="https://stackpath.bootstrapcdn.com/bootstrap/4.5.2/js/bootstrap.min.js"></script>
</head>
<body>
    {%app_entry%}
    <footer>
        {%config%}
        {%scripts%}
        {%renderer%}
    </footer>
</body>
</html>
"""

In [8]:
#NAVBAR

NAVBAR =     html.Nav(className="navbar navbar-expand-lg navbar-light bg-light", style={'height': '97px'}, children=[
        html.A(href="https://www.nvent.com/en-us/", className="navbar-brand", children=[
            html.Img(src="https://www.nvent.com/themes/custom/particle/dist/app-drupal/assets/images/logo-nvent.svg", height="50px")
        ]),
        html.Button(className="navbar-toggler", type="button", **{'data-toggle': 'collapse', 'data-target': '#navbarNav'}, children=[
            html.Span(className="navbar-toggler-icon")
        ]),
        html.Div(className="collapse navbar-collapse", id="navbarNav", children=[
            html.Ul(className="navbar-nav mr-auto", children=[
                html.Li(className="nav-item", children=[
                    html.A("HOME", className="nav-link", href="/")
                ]),
                html.Li(className="nav-item dropdown", children=[
                    html.A("REPORTS", className="nav-link dropdown-toggle", href="#", id="navbarDropdown", role="button", **{'data-toggle': 'dropdown'}, **{'aria-haspopup': 'true', 'aria-expanded': 'false'}),
                    html.Div(className="dropdown-menu", **{'aria-labelledby': 'navbarDropdown'}, children=[
                        html.A("TEST REPORT", className="dropdown-item", href="/test-report"),
                        html.A("OPERATOR REPORT", className="dropdown-item", href="/operator-report"),
                        html.A("CELL REPORT", className="dropdown-item", href="/cell-report"),
                        html.A("REFRIGERANT REPORT", className="dropdown-item", href="/refrigerant-report"),
                        html.A("FLUID REPORT", className="dropdown-item", href="/fluid-report")
                    ])
                ]),
                html.Li(className="nav-item", children=[
                    html.A("LAB DOCUMENTATION", className="nav-link", href="/lab-documentation")
                ]),
                html.Li(className="nav-item", children=[
                    html.A("DATABASE", className="nav-link", href="/database")
                ])
            ]),
            html.Form(className="form-inline my-2 my-lg-0", children=[
                dcc.Input(type="search", placeholder="Search", className="form-control mr-sm-2"),
                html.Button("Search", className="btn btn-outline-success my-2 my-sm-0")
            ])
        ])
    ])

In [9]:
# CONTENT

CONTENT = html.Div(id='page-content', style={
        'minHeight': '560px',
        'borderRadius': '5px',
        'backgroundColor': '#f8f9fa'
    })

In [10]:
#FOOTER 
FOOTER = html.Footer(className='footer bg-light text-center text-lg-start', children=[
        html.Div(className='text-center p-3', style={'backgroundColor': 'rgba(0, 0, 0, 0.2)'}, children=[
            "© 2023 Laboratory Dashboard"
        ])
    ])

In [11]:
#DATABASE PAGE

DATABASE =  html.Div(dash_table.DataTable(
                id='datatable-row-ids',
                columns=[
                    {'name': i, 'id': i, 'deletable': True} for i in database.columns #DELETED?
                    if i != 'id'
                ],
                data=data,
                #editable=True,
                filter_action="native",
                sort_action="native",
                sort_mode='multi',
                row_selectable='multi',
                selected_rows=[],
                page_action='native',
                page_current=0,
                page_size=50,
                style_table={'overflowX': 'auto'},  # Enable horizontal scrolling
                style_cell={
                    'minWidth': '150px', 'width': '150px', 'maxWidth': '2000px',  # Set cell width
                    'textAlign': 'center'  # Centralize text in cells
                },  # Set cell width
            ))
        

In [12]:
# REPORT PAGE

TEST_REPORT = html.Div([
                    # First column (size 3)
                    html.Div([
                    # Card for dropdowns and submit button
                            html.Div(className='card mx-1', style={'backgroundColor': '#f8f9fa', 'margin': '5px'}, children=[
                                html.Div(className='card-header', children=[
                                    html.H6('Test selection')
                                ]),
                                html.Div(className='card-body', children=[
                                    # Test status checkbox (Passed) at the top
                                    dcc.Checklist(
                                        id='test-status-checkbox',
                                        options=[{'label': 'Passed', 'value': 'passed'}],
                                        value=[],
                                        style={'marginBottom': '20px'}  # Increase space below the checklist
                                    ),
                                    # Part number dropdown
                                    dcc.Dropdown(
                                        id='part-number-dropdown',
                                        options=[{'label': part, 'value': part} for part in database['Part number'].dropna().unique() if part],
                                        placeholder='Select Part Number',
                                        style={'marginBottom': '20px'}  # Increase space below the dropdown
                                    ),
                                    # Test ID dropdown
                                    dcc.Dropdown(
                                        id='test-id-dropdown',
                                        placeholder='Select Test ID',
                                        style={'marginBottom': '20px'}  # Increase space below the dropdown
                                    ),
                                    # Test number dropdown
                                    dcc.Dropdown(
                                        id='test-number-dropdown',
                                        placeholder='Select Test Number',
                                        style={'marginBottom': '10px'}  # Increase space below the dropdown
                                    ),
                                    # Submit button with increased distance from the dropdowns
                                    html.Button('Submit', id='submit-button', n_clicks=0, style={'margin-top': '10px'})
                                ])
                            ]),
                    # Card for test information___________________________________________________________________________________
                    html.Div(className='card mx-1', style={'backgroundColor': '#f8f9fa', 'margin': '5px', 'height': '67%'}, children=[
                        html.Div(className='card-body', id='logo', children=[
                            html.Img(src='/assets/Nvent.jpg', style={'width': '100%', 'height': 'auto'})
                        ])
                    ])
                    #_____________________________________________________________________________________________________________________ END CARD TEST INFORMATION

                ], className='col-3'),
# Second column (size 9)
html.Div(id='testa-tables', className='col-9', children=[
    html.Div(className='row', style={'margin-right': '0px', 'margin-left': '-35px', 'display': 'flex'}, children=[
        html.Div(className='col-5', style={'padding-right': '0px', 'padding-left': '2px', 'margin-right': '0px', 'display': 'flex', 'flex-direction': 'column'}, children=[
            html.Div(className='card mx-1', style={'backgroundColor': '#f8f9fa', 'margin': '5px', 'marginRight': '0px', 'flex': '1'}, children=[
                html.Div(className='card-header', children=[
                    html.H6('Test Information')
                ]),
                html.Div(className='card-body', id='test-anagraph-content', children=[
                    # Initial empty table
                    html.Table(className='table', children=[
                        html.Tbody([
                            #_____________________________________________
                            html.Tr([html.Td("Test ID: "), html.Td()]),
                            html.Tr([html.Td("Order: "), html.Td()]),
                            html.Tr([html.Td("Test number: "), html.Td()]),
                            html.Tr([html.Td("Date: ", style={'border-bottom': '3px solid grey'}), html.Td(style={'border-bottom': '3px solid grey'})]),
                            #______________________________________________
                            html.Tr([html.Td("Part number: "), html.Td()]),
                            html.Tr([html.Td("Revision: "), html.Td()]),
                            html.Tr([html.Td("Serial number: ", style={'border-bottom': '3px solid grey'}), html.Td(style={'border-bottom': '3px solid grey'})]),
                            #_______________________________________________
                            html.Tr([html.Td("Test bench: "), html.Td()]),
                            html.Tr([html.Td("Operator: "), html.Td()]),
                            html.Tr([html.Td("Hydraulic line: ", style={'border-bottom': '3px solid grey'}), html.Td(style={'border-bottom': '3px solid grey'})]),
                            #_______________________________________________
                            html.Tr([html.Td("Fluid: "), html.Td()]),
                            html.Tr([html.Td("Circuits: "), html.Td()]),  
                            html.Tr([html.Td("Refrigerant: "), html.Td()]),
                            html.Tr([html.Td("Configuration: ", style={'border-bottom': '3px solid grey'}), html.Td(style={'border-bottom': '3px solid grey'})]),
                            #_______________________________________________
                            html.Tr([html.Td("Duration: "), html.Td()]),
                            html.Tr([html.Td("Log file: "), html.Td()]),
                            html.Tr([html.Td("Last modified: ", style={'border-bottom': '3px solid grey'}), html.Td(style={'border-bottom': '3px solid grey'})]),
                            #_______________________________________________
                            html.Tr([html.Td("Status: "), html.Td()]),
                            html.Tr([html.Td("Approved by: "), html.Td()])
                        ])
                    ])
                ])
            ])
        ]),
        # 1st row with card "test results"
        html.Div(className='col-7', style={'padding-right': '0px', 'padding-left': '0px', 'margin-left': '-3px', 'display': 'flex', 'flex-direction': 'column'}, children=[
            html.Div(className='card mx-1', style={'backgroundColor': '#f8f9fa', 'margin': '5px','margin-left': '-40px', 'flex': '1'}, children=[
                html.Div(className='card-header', children=[
                    html.H6('Test Results')
                ]),
                html.Div(className='card-body', id='test-results-content', children=[
                    # Initial empty table
                    html.Table(className='table', children=[
                        html.Thead([
                            html.Tr([html.Th("Index"), html.Th("Parameter"), html.Th("Unit"), html.Th("Circuit 1"), html.Th("Circuit 2")])
                        ]),
                        html.Tbody([
                            html.Tr([html.Td("1"), html.Td("Condensation pressure"), html.Td("bar"), html.Td(), html.Td()]),
                            html.Tr([html.Td("2"), html.Td("Condensation temperature"), html.Td("°C"), html.Td(), html.Td()]),
                            html.Tr([html.Td("3"), html.Td("Compressor inlet temperature"), html.Td("°C"), html.Td(), html.Td()]),
                            html.Tr([html.Td("4"), html.Td("Compressor outlet temperature"), html.Td("°C"), html.Td(), html.Td()]),
                            html.Tr([html.Td("5"), html.Td("Evaporation pressure"), html.Td("bar"), html.Td(), html.Td()]),
                            html.Tr([html.Td("6"), html.Td("Evaporation temperature"), html.Td("°C"), html.Td(), html.Td()]),
                            html.Tr([html.Td("7"), html.Td("Subcooling"), html.Td("°C"), html.Td(), html.Td()]),
                            html.Tr([html.Td("8"), html.Td("Superheating"), html.Td("°C"), html.Td(), html.Td()]),
                            html.Tr([html.Td("9"), html.Td("Ambient temperature"), html.Td("°C"), html.Td(), html.Td()]),
                            html.Tr([html.Td("10"), html.Td("Inlet fluid temperature"), html.Td("°C"), html.Td(), html.Td()]),
                            html.Tr([html.Td("11"), html.Td("Outlet fluid temperature"), html.Td("°C"), html.Td(), html.Td()]),
                            html.Tr([html.Td("12"), html.Td("Delta"), html.Td("°C"), html.Td(), html.Td()]),
                            html.Tr([html.Td("13"), html.Td("Flow rate"), html.Td("L/min"), html.Td(), html.Td()]),
                            html.Tr([html.Td("14"), html.Td("Cooling capacity"), html.Td("kW"), html.Td(), html.Td()]),
                            html.Tr([html.Td("15"), html.Td("Tension"), html.Td("V"), html.Td(), html.Td()]),
                            html.Tr([html.Td("16"), html.Td("Current"), html.Td("A"), html.Td(), html.Td()]),
                            html.Tr([html.Td("17"), html.Td("Power"), html.Td("kW"), html.Td(), html.Td()])
                        ])
                    ])
                ])
            ])
        ])
        # End card test result
    ])
])
# End report content

    ], className='row', style={'minHeight': '560px', 'borderRadius': '5px', 'backgroundColor': '#ffffff', 'overflowX': 'auto', 'overflowY': 'auto'})

In [18]:
# CELL REPORT AND MENUS DATA 

test_data_new = pd.read_excel('Plan.xlsx', header = 0, sheet_name = 'TEST') # Import data from new database
test_data_new.dropna(how='all')
test_data_new.dropna(axis=1, how='all', inplace=True)

cells_calendar = pd.read_excel('Plan.xlsx', header = 0, sheet_name = 'CELLS') # Import data - Cells calendar

# SETTIME PARAMETERS AS DATETIME

test_data_new['STIMATION'] =  pd.to_timedelta(test_data_new['STIMATION'], unit='h')
test_data_new['DURATION'] =  pd.to_timedelta(test_data_new['DURATION'], unit='h')
test_data_new['DUE.DATE'] = pd.to_datetime(test_data_new['DUE.DATE']) #infer_datetime_format=True
test_data_new.sort_values(by='DUE.DATE', ascending=False)
test_data_new['DUE.DATE'] = pd.to_datetime(test_data_new['DUE.DATE'], errors='coerce').dt.strftime('%d/%m/%Y')
test_data_new['EXPECTED START'] = pd.to_datetime(test_data_new['EXPECTED START'], errors='coerce').dt.strftime('%d/%m/%Y')
test_data_new['P_N'] = test_data_new['P_N'].str.replace(' ', '')
# Format 'STIMATION' to numeric value in hours
test_data_new['STIMATION'] = test_data_new['STIMATION'].apply(lambda x: x.total_seconds() / 3600)

status_table = test_data_new[['CELL', 'TEST.ID', "P_N", "EXPECTED START", "DUE.DATE", "TEST.DESCRIPTION", "TEST.STATUS", 'STIMATION', 'PRIORITY']]

# Define the table headings
table_headings = ["Testing room", "Test ID", "Part number", "Priority", "Due date", "Test description", "Status", "Stimation"]

#DYNAMICTABLE
dynamic_table = dash_table.DataTable(
    columns=[
        {'name': 'Room', 'id': 'CELL'},
        {'name': 'Test ID', 'id': 'TEST.ID'},
        {'name': 'Part number', 'id': 'P_N'},
        {'name': 'Priority', 'id': 'PRIORITY', "editable": True},
        {'name': 'Start', 'id': "EXPECTED START", "editable": True},
        {'name': 'Due date', 'id': 'DUE.DATE', "editable": True},
        {'name': 'Test description', 'id': 'TEST.DESCRIPTION'},
        {'name': 'Status', 'id': 'TEST.STATUS', 'presentation': 'dropdown', "editable": True},
        {'name': 'Stimation (h)', 'id': 'STIMATION', "editable": True}
    ],
    page_size=10,
    fixed_rows={'headers': True},
    style_table={'height': '300px', 'overflowY': 'auto'},
    style_cell={
        'textAlign': 'left',
        'font-family': 'Arial, sans-serif',
        'font-size': '14px',
        'padding': '5px'
    },
    style_header={
        'textAlign': 'center',
        'font-family': 'Arial, sans-serif',
        'font-size': '14px',
        'font-weight': 'bold',
        'backgroundColor': '#f8f9fa',
        'color': '#333'
    },
    filter_action="native",
    sort_action="native",
    selected_rows=list(range(10)),  # Select the first 10 rows by default
    sort_mode='multi',
    row_selectable="multi",
    style_data_conditional=[
        {
            'if': {
                'column_id': 'DUE.DATE',
            },
            'textAlign': 'center',
            'width': '60px'
        },
        {
            'if': {
                'column_id': 'EXPECTED START',
            },
            'textAlign': 'center',
            'width': '60px'
        },
        {
            'if': {
                'column_id': 'P_N',
            },
            'textAlign': 'center',
            'width': '60px'
        },
        {
            'if': {
                'column_id': 'TEST.ID',
            },
            'textAlign': 'center',
            'width': '40px'
        },
        {
            'if': {
                'column_id': 'TEST.STATUS',
            },
            'textAlign': 'center',
            'width': '40px'
        },
        {
            'if': {
                'column_id': 'STIMATION',
            },
            'textAlign': 'center',
            'width': '80px'
        },
        {
            'if': {
                'column_id': 'PRIORITY',
            },
            'textAlign': 'center',
            'width': '50px'
        },
        {
            'if': {
                'column_id': 'TEST.DESCRIPTION',
            },
            'width': '30px'
        },
        {
            'if': {
                'column_id': 'CELL',
            },
            'textAlign': 'center',
            'width': '50px'
        }
    ],
    id="management-table-draft",
    data=status_table.to_dict("records"),
    dropdown={
        'TEST.STATUS': {
            'options': [
                {'label': 'COMPLETED', 'value': 'COMPLETED'},
                {'label': 'ON GOING', 'value': 'ON GOING'},
                {'label': 'HOLDING', 'value': 'HOLDING'},
                {'label': 'CANCELED', 'value': 'CANCELED'},
                {'label': 'PAUSED', 'value': 'PAUSED'}
            ]
        }
    },
    style_as_list_view=True,
    css = [{
    "selector": ".Select-menu-outer",
    "rule": 'display : block !important'
}]
)




c:\Users\e2023898\AppData\Local\Programs\Python\Python312\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning:

Data Validation extension is not supported and will be removed



In [14]:
# Create Gantt chart data
gantt_data = []
for _, row in status_table.iterrows():
    gantt_data.append(dict(
        Task=row['TEST.ID'],
        Start=row['EXPECTED START'],
        Finish=row['DUE.DATE'],
        Resource=row['CELL']
    ))

# Create Gantt chart

GANNT = dbc.CardBody(
                dcc.Graph(id='gantt-chart')
                )

In [20]:
CELLREPORT = html.Div([
    html.Div([
        # First card with selectors
        html.Div(className='card mx-1', style={'backgroundColor': '#f8f9fa', 'margin': '2px'}, children=[
            html.Div(className='card-header', children=[
                html.H6('Filters')
            ]),
            html.Div(className='card-body', children=[
                # Date picker range at the top
                dcc.DatePickerRange(
                    id='date-picker-range',
                    start_date_placeholder_text='Start Date',
                    end_date_placeholder_text='End Date',
                    display_format='DD/MM/YYYY',  # Display format for the date picker
                    style={'marginBottom': '20px', 'backgroundColor': '#f8f9fa', 'color': '#333'}  # Adjust colors
                ),
                # Group of checkboxes with options (R60, R110)
                dcc.Checklist(
                    id='selected_cell',
                    options=[
                        {'label': 'R60', 'value': 60},
                        {'label': 'R110', 'value': 110}
                    ],
                    value=[],
                    style={'marginBottom': '20px'}  # Increase space below the checklist
                ),
                # Status menu with options (COMPLETED, ON GOING, HOLDING, CANCELED)
                dcc.Dropdown(
                    id='status_cell',
                    options=[
                        {'label': 'COMPLETED', 'value': 'COMPLETED'},
                        {'label': 'ON GOING', 'value': 'ON GOING'},
                        {'label': 'HOLDING', 'value': 'HOLDING'},
                        {'label': 'CANCELED', 'value': 'CANCELED'},
                        {'label': 'PAUSED', 'value': 'PAUSED'},
                        {'label': 'ALL', 'value': 'ALL'}  # Add option to see all table entries
                    ],
                    placeholder='Select Status',
                    style={'marginBottom': '20px'}  # Increase space below the dropdown
                ),
                # Submit button with increased distance from the dropdowns
                html.Button('Submit', id='submit-button', n_clicks=0, style={'width': '200px', 'height': '30px', 'margin': '5px'}),
                html.Button("Reset", id="reset-button", n_clicks=0, style={'width': '200px', 'height': '30px', 'margin': '5px'}),

            ])
        ]),
        # Second empty card
        html.Div(className='card mx-1', style={'backgroundColor': '#f8f9fa', 'margin': '1px'}, children=[
            html.Div(className='card-body', children=[                # Add the submit button for the chart
                html.Button('Update Chart', id='update-chart-button', n_clicks=0, style={'width': '200px', 'height': '30px', 'margin': '5px'}),
                # Add the reset button for the chart
                html.Button('Reset Chart', id='reset-chart-button', n_clicks=0, style={'width': '200px', 'height': '30px', 'margin': '5px'})])
        ]),
        # Third empty card
        html.Div(className='card mx-1', style={'backgroundColor': '#f8f9fa', 'margin': '1px'}, children=[
            html.Div(className='card-body', children=[])
        ])
    ], className='col-3'),
    # Second column with the table
    html.Div(id='management-table', className='col-9', children=[
        dbc.Card(
            [
                dbc.CardBody(
                    [                    
                    html.Button('Select All', id='select-all-button', n_clicks=0, style={'width': '100px', 'height': '30px', 'margin': '5px'}),
                    html.Button('Deselect All', id='deselect-all-button', n_clicks=0, style={'width': '100px', 'height': '30px', 'margin': '5px'}),
                    dynamic_table,
                    html.Button('Update Database', id='update-database-button', n_clicks=0, style={'width': '150px', 'height': '30px', 'margin': '5px'})
                    ]
                )
            ],
            style={"margin": "1px", 'marginLeft':'-25px', 'marginRight':'-15px'}  # Adjust margins
        ),
        dbc.Card([GANNT],
            style={"margin": "1px", 'marginLeft':'-25px', 'marginRight':'-15px', 'flex': '1'}  # Adjust margins
        )
    ])
], className='row', style={'margin': '0px', 'marginLeft':'-20px'})  # Adjust margins between the columns
          

In [32]:
TABLE_template = """
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <title>Laboratory Dashboard</title>
    <link rel="stylesheet" href="https://stackpath.bootstrapcdn.com/bootstrap/4.5.2/css/bootstrap.min.css">
    <link rel="stylesheet" href="https://cdn.datatables.net/1.11.3/css/jquery.dataTables.min.css">
    <script src="https://code.jquery.com/jquery-3.6.0.min.js"></script>
    <script src="https://cdn.datatables.net/1.11.3/js/jquery.dataTables.min.js"></script>
</head>
<body>
    <nav class="navbar navbar-expand-lg navbar-light bg-light" style="height: 97px;">
        <!-- Navbar content here -->
    </nav>
    <div class="container">
        <div>
            <label for="start-date">Start Date:</label>
            <input type="date" id="start-date">
            <label for="end-date">End Date:</label>
            <input type="date" id="end-date">
            <label for="cell-selector">Cell:</label>
            <select id="cell-selector">
                <option value="60">R60</option>
                <option value="110">R110</option>
            </select>
            <label for="status-selector">Status:</label>
            <select id="status-selector">
                <option value="ALL">ALL</option>
                <option value="COMPLETED">COMPLETED</option>
                <option value="ON GOING">ON GOING</option>
                <option value="HOLDING">HOLDING</option>
                <option value="CANCELED">CANCELED</option>
                <option value="PAUSED">PAUSED</option>
            </select>
            <button id="filter-button">Filter</button>
            <button id="reset-button">Reset</button>
        </div>
        <table id="dynamic-table" class="display">
            <thead>
                <tr>
                    <th>Room</th>
                    <th>Test ID</th>
                    <th>Part number</th>
                    <th>Priority</th>
                    <th>Start</th>
                    <th>Due date</th>
                    <th>Test description</th>
                    <th>Status</th>
                    <th>Stimation (h)</th>
                </tr>
            </thead>
            <tbody></tbody>
        </table>
        <button id="update-button">Update Database</button>
    </div>
    <footer>
        <!-- Footer content here -->
    </footer>
    <script>
        $(document).ready(function() {
            var table = $('#dynamic-table').DataTable({
                ajax: {
                    url: '/data',
                    dataSrc: ''
                },
                columns: [
                    { data: 'CELL' },
                    { data: 'TEST.ID' },
                    { data: 'P_N' },
                    { data: 'PRIORITY', editable: true },
                    { data: 'EXPECTED START', editable: true },
                    { data: 'DUE.DATE', editable: true },
                    { data: 'TEST.DESCRIPTION' },
                    { data: 'TEST.STATUS', editable: true },
                    { data: 'STIMATION', editable: true }
                ]
            });
            $('#filter-button').click(function() {
                var startDate = $('#start-date').val();
                var endDate = $('#end-date').val();
                var cell = $('#cell-selector').val();
                var status = $('#status-selector').val();
                table.columns(4).search(startDate).draw();
                table.columns(5).search(endDate).draw();
                table.columns(0).search(cell).draw();
                table.columns(7).search(status).draw();
            });
            $('#reset-button').click(function() {
                $('#start-date').val('');
                $('#end-date').val('');
                $('#cell-selector').val('');
                $('#status-selector').val('ALL');
                table.columns().search('').draw();
            });
            $('#update-button').click(function() {
                var data = table.rows().data().toArray();
                $.ajax({
                    url: '/update',
                    type: 'POST',
                    contentType: 'application/json',
                    data: JSON.stringify(data),
                    success: function(response) {
                        alert('Database updated successfully');
                    }
                });
            });
        });
    </script>
</body>
</html>
"""

In [35]:
app = Flask(__name__)

# Load the data from the Excel file
def load_data():
    test_data_new = pd.read_excel('Plan.xlsx', header=0, sheet_name='TEST')
    test_data_new.dropna(how='all', inplace=True)
    test_data_new.dropna(axis=1, how='all', inplace=True)
    test_data_new['STIMATION'] = pd.to_timedelta(test_data_new['STIMATION'], unit='h')
    test_data_new['DURATION'] = pd.to_timedelta(test_data_new['DURATION'], unit='h')
    test_data_new['DUE.DATE'] = pd.to_datetime(test_data_new['DUE.DATE'])
    test_data_new.sort_values(by='DUE.DATE', ascending=False, inplace=True)
    test_data_new['DUE.DATE'] = test_data_new['DUE.DATE'].dt.strftime('%d/%m/%Y')
    test_data_new['EXPECTED START'] = pd.to_datetime(test_data_new['EXPECTED START']).dt.strftime('%d/%m/%Y')
    test_data_new['P_N'] = test_data_new['P_N'].str.replace(' ', '')
    test_data_new['STIMATION'] = test_data_new['STIMATION'].apply(lambda x: x.total_seconds() / 3600)
    return test_data_new

@app.route('/')
def index():
    return render_template('index.html')

@app.route('/data')
def data():
    test_data_new = load_data()
    status_table = test_data_new[['CELL', 'TEST.ID', 'P_N', 'EXPECTED START', 'DUE.DATE', 'TEST.DESCRIPTION', 'TEST.STATUS', 'STIMATION', 'PRIORITY']]
    return jsonify(status_table.to_dict(orient='records'))

@app.route('/update', methods=['POST'])
def update():
    data = request.json
    test_data_new = load_data()
    updated_df = pd.DataFrame(data)
    updated_df.set_index('TEST.ID', inplace=True)
    test_data_new.set_index('TEST.ID', inplace=True)
    merged_df = test_data_new.combine_first(updated_df).reset_index()
    merged_df.to_excel('Plan.xlsx', sheet_name='TEST', index=False)
    return jsonify({'status': 'success'})

if __name__ == '__main__':
    app.run(debug=True)

 * Serving Flask app '__main__'
 * Debug mode: on


SystemExit: 1

In [37]:
from flask import Flask

app = Flask(__name__)

@app.route('/')
def index():
    return "Hello, World!"

if __name__ == '__main__':
    app.run(debug=True)

 * Serving Flask app '__main__'
 * Debug mode: on


SystemExit: 1

C:\Users\e2023898\AppData\Roaming\Python\Python312\site-packages\IPython\core\interactiveshell.py:3585: UserWarning:

To exit: use 'exit', 'quit', or Ctrl-D.



In [36]:
# Initialize the Dash app

app = dash.Dash(__name__, external_stylesheets=[
    'https://stackpath.bootstrapcdn.com/bootstrap/4.5.2/css/bootstrap.min.css'
])
app.config.suppress_callback_exceptions = True
app.index_string = html_template



# Define the layout of the app
app.layout = html.Div([
    dcc.Location(id='url', refresh=False), NAVBAR, CONTENT, FOOTER,  
])

# Define the callback to update the page content based on navigation
@app.callback(
    Output('page-content', 'children'),
    [Input('url', 'pathname')]
)
def display_page(pathname):
    content_style = {
        'minHeight': '560px',
        'borderRadius': '5px',
        'border': '1px solid red',
        'margin': '10px',
        'padding': '10px',
        'backgroundColor': '#ffffff',
        'overflowX': 'auto',  # Enable horizontal scrolling
        'overflowY': 'auto'   # Enable vertical scrolling
    }
    
    if pathname == '/':
        return html.Div([
            html.H1("Welcome to the Laboratory Dashboard")
        ], style=content_style)
#TEST REPORT_________________________________________________________________________________________________________
    elif pathname == '/test-report': # 
        return TEST_REPORT
#END TEST REPORT_________________________________________________________________________________________________________
    elif pathname == '/operator-report':
        return html.Div([
            html.H1("Operator Report Page")
        ], style=content_style)
# CELL REPORT__________#####################################################################################################
    elif pathname == '/cell-report':
        return html.Div([CELLREPORT] , style=content_style)
#END CELL REPORT ____________________________________________________________________________________________________________
    elif pathname == '/refrigerant-report':
        return html.Div([
            html.H1("Refrigerant Report Page")
        ], style=content_style)
    elif pathname == '/fluid-report':
        return html.Div([
            html.H1("Fluid Report Page")
        ], style=content_style)
    elif pathname == '/lab-documentation':
        return html.Div([
            html.H1("Lab Documentation Page")
        ], style=content_style)
#DATABASE_________________________________________________________________________________________________
    elif pathname == '/database':
 # Convert timedelta to string before returning
        data = database.to_dict('records')
        for record in data:
            for key, value in record.items():
                if isinstance(value, datetime.timedelta):
                    record[key] = timedelta_to_str(value)
        return html.Div([DATABASE], style=content_style)
    else:
        return html.Div([
            html.H1("404 Page Not Found")
        ], style=content_style)
#END DATABASE________________________________________________________________________________________________________________
# Callback to update the Test ID dropdown based on selected Part Number
@app.callback(
    Output('part-number-dropdown', 'options'),
    [Input('test-status-checkbox', 'value')],
    allow_duplicate=True
)
def update_part_number_dropdown(test_status):
    if 'passed' in test_status:
        filtered_df = database[database['Test status'] == 'PASSED']
        part_numbers = filtered_df['Part number'].dropna().unique()
        return [{'label': part_number, 'value': part_number} for part_number in part_numbers]
    else:
        part_numbers = database['Part number'].dropna().unique()
        return [{'label': part_number, 'value': part_number} for part_number in part_numbers]
    
@app.callback(
    Output('test-id-dropdown', 'options'),
    [Input('part-number-dropdown', 'value'), Input('test-status-checkbox', 'value')],
    allow_duplicate=True
)
def update_test_id_dropdown(part_number, test_status):
    if part_number:
        filtered_df = database[database['Part number'] == part_number]
        if 'passed' in test_status:
            filtered_df = filtered_df[filtered_df['Test status'] == 'PASSED']
        test_ids = filtered_df['Test ID'].dropna().unique()
        return [{'label': test_id, 'value': test_id} for test_id in test_ids]
    return []

@app.callback(
    Output('test-number-dropdown', 'options'),
    [Input('part-number-dropdown', 'value'), Input('test-id-dropdown', 'value'), Input('test-status-checkbox', 'value')],
    allow_duplicate=True
)
def update_test_number_dropdown(part_number, test_id, test_status):
    if part_number and test_id:
        filtered_df = database[(database['Part number'] == part_number) & (database['Test ID'] == test_id)]
        if 'passed' in test_status:
            filtered_df = filtered_df[filtered_df['Test status'] == 'PASSED']
        test_numbers = filtered_df['Test number'].dropna().unique()
        return [{'label': test_number, 'value': test_number} for test_number in test_numbers]
    return []

#CALL BACK TEST INFO_______________________________________________

@app.callback(
    [Output('test-anagraph-content', 'children'),
     Output('test-results-content', 'children')],
    [Input('submit-button', 'n_clicks')],
    [State('part-number-dropdown', 'value'), State('test-id-dropdown', 'value'), State('test-number-dropdown', 'value')]
)
def update_content(n_clicks, part_number, test_id, test_number):
    if n_clicks > 0 and part_number and test_id and test_number:
        filtered_df = database[(database['Part number'] == part_number) & 
                               (database['Test ID'] == test_id) & 
                               (database['Test number'] == test_number)]
        if not filtered_df.empty:
            row = filtered_df.iloc[0]
            status = row['Test status'] if pd.notna(row['Test status']) else "Info not available"
            date_value = pd.to_datetime(row['Date']).strftime('%d/%m/%Y') if pd.notna(row['Date']) else "Info not available"
            
            anagraph_content = html.Table(className='table', children=[
                html.Tbody([
                    html.Tr([html.Td("Order: "), html.Td(row['Order'])]),
                    html.Tr([html.Td("Test ID: "), html.Td(row['Test ID'])]),
                    html.Tr([html.Td("Test number: "), html.Td(row['Test number'])]),
                    html.Tr([html.Td("Date: ", style={'border-bottom': '3px solid grey'}), html.Td(date_value, style={'border-bottom': '3px solid grey'})]),
                    
                    html.Tr([html.Td("Part number: "), html.Td(row['Part number'])]),
                    html.Tr([html.Td("Revision: "), html.Td(row['Revision'])]),
                    html.Tr([html.Td("Serial number: ", style={'border-bottom': '3px solid grey'}), html.Td(row['Serial number'], style={'border-bottom': '3px solid grey'})]),

                    html.Tr([html.Td("Test bench: "), html.Td(row['Cell'])]),
                    html.Tr([html.Td("Operator: "), html.Td(row['Operator'])]),
                    html.Tr([html.Td("Hydraulic line: ", style={'border-bottom': '3px solid grey'}), html.Td(row['Feeding line'], style={'border-bottom': '3px solid grey'})]),
                    #_______________________________________________
                    html.Tr([html.Td("Fluid: "), html.Td(row['Fluid'])]),
                    html.Tr([html.Td("Circuits: "), html.Td(row['Circuits'])]),  
                    html.Tr([html.Td("Refrigerant: "), html.Td(row['Refrigerant'])]),
                    html.Tr([html.Td("Configuration: ", style={'border-bottom': '3px solid grey'}), html.Td(row['Note'], style={'border-bottom': '3px solid grey'})]),
                     #_______________________________________________
                    html.Tr([html.Td("Duration: "), html.Td(row ['Duration'])]),
                    html.Tr([html.Td("Log file: "), html.Td(row['File name'])]),
                    html.Tr([html.Td("Last modified: ", style={'border-bottom': '3px solid grey'}), html.Td(row['Last modified'], style={'border-bottom': '3px solid grey'})]),
                     #______________________________________________
                    html.Tr([html.Td("Approved by: "), html.Td(row['Manager'])]),
                    html.Tr([html.Td("Status: "), html.Td(status)])
                ])
            ])
            
            fluid = row['Fluid']
            presa_elettrica = row['Presa elettrica selezionata 1..5 da pann. Operatore']

            def get_t2_by_fluid():
                if fluid == 'OIL':
                    return row["Temperatura media uscita evaporatore 1"]
                elif fluid == 'WATER':
                    return row["Temperatura media uscita evaporatore 2"]
                else:
                    return row["Temperatura media uscita evaporatore 3"]

            def get_t1_by_fluid():
                if fluid == 'OIL':
                    return row["Temperatura media ingresso evaporatore 1"]
                elif fluid == 'WATER':
                    return row["Temperatura media ingresso evaporatore 2"]
                else:
                    return row["Temperatura media ingresso evaporatore 3"]

            def get_flow_by_fluid():
                if fluid == 'OIL':
                    return row["Portata totale evaporatore A"]
                elif fluid == 'WATER':
                    return row["Portata totale evaporatore B"]
                else:
                    return row["Portata totale evaporatore C"]
            
            def get_resa_by_fluid():
                if fluid == 'OIL':
                    return row["Resa lato evaporatore 1 (Olio)"]
                elif fluid == 'WATER':
                    return row["Resa lato evaporatore 2 (Acqua)"]
                else:
                    return row["Resa lato evaporatore 2 (Glycol)"]
            
            def get_value_by_presa_elettrica():
                if presa_elettrica == 0:
                    return row['Tensione concatenata media - Presa 1']
                elif presa_elettrica == 1:
                    return row['Tensione concatenata media - Presa 2']
                elif presa_elettrica == 2:
                    return row['Tensione concatenata media - Presa 3']
                elif presa_elettrica == 3:
                    return row['Tensione concatenata media - Presa 4']
                elif presa_elettrica == 4:
                    return row['Tensione concatenata media - Presa 5']
                else:
                    return "Info not available"
                
            def get_current_by_presa_elettrica():
                if presa_elettrica == 0:
                    return row["Corrente totale - Presa 1"]
                elif presa_elettrica == 1:
                    return row["Corrente totale - Presa 2"]
                elif presa_elettrica == 2:
                    return row["Corrente totale - Presa 3"]
                elif presa_elettrica == 3:
                    return row["Corrente totale - Presa 4"]
                elif presa_elettrica == 4:
                    return row["Corrente totale - Presa 5"]
                else:
                    return "Info not available"
                
            def get_power_by_presa_elettrica():
                if presa_elettrica == 0:
                    return row["Potenza Attiva Totale - Presa 1"]
                elif presa_elettrica == 1:
                    return row["Potenza Attiva Totale - Presa 2"]
                elif presa_elettrica == 2:
                    return row["Potenza Attiva Totale - Presa 3"]
                elif presa_elettrica == 3:
                    return row["Potenza Attiva Totale - Presa 4"]
                elif presa_elettrica == 4:
                    return row["Potenza Attiva Totale - Presa 5"]
                else:
                    return "Info not available"
                
            delta =  abs(round((pd.to_numeric(get_t2_by_fluid()) - pd.to_numeric(get_t1_by_fluid())),2))
            
            table_rows = [
                html.Tr([html.Td("1"), html.Td("Condensation pressure"), html.Td("bar"), html.Td(row['Pressione Condensazione 1']), html.Td(row['Pressione Condensazione 2'])]),
                html.Tr([html.Td("2"), html.Td("Condensation temperature"), html.Td("°C"), html.Td(row['Temperatura condensazione 1 calcolata (da P7)']), html.Td(row['Temperatura condensazione 2 calcolata (da P8)'])]),
                html.Tr([html.Td("3"), html.Td("Compressor inlet temperature"), html.Td("°C"), html.Td(row['Temperatura Aspirazione 1']), html.Td(row['Temperatura Aspirazione 2'])]),
                html.Tr([html.Td("4"), html.Td("Compressor outlet temperature"), html.Td("°C"), html.Td(row['Temperatura Liquida 1']), html.Td(row['Temperatura Liquida 2'])]),
                html.Tr([html.Td("5"), html.Td("Evaporation pressure"), html.Td("bar"), html.Td(row['Pressione Evaporazione 1']), html.Td(row['Pressione Evaporazione 2'])]),
                html.Tr([html.Td("6"), html.Td("Evaporation temperature"), html.Td("°C"), html.Td(row['Temperatura evaporazione 1 calcolata (da P9)']), html.Td(row['Temperatura evaporazione 2 calcolata (da P10)'])]),
                html.Tr([html.Td("7"), html.Td("Subcooling"), html.Td("°C"), html.Td(row['Sottoraffreddamento 1 (TC7-TC_P7)']), html.Td(row['Sottoraffreddamento 2 (TC8-TC_P8)'])]),
                html.Tr([html.Td("8"), html.Td("Superheating"), html.Td("°C"), html.Td(row['Surriscaldamento 1 (TC9-TC_P9)']), html.Td(row['Surriscaldamento 2 (TC10-TC_P10)'])]),
                html.Tr([html.Td("9"), html.Td("Ambient temperature"), html.Td("°C"), html.Td(row['Temperatura aria cabina media']), html.Td()]),
                html.Tr([html.Td("10"), html.Td("Inlet fluid temperature"), html.Td("°C"), html.Td(get_t1_by_fluid()), html.Td()]),
                html.Tr([html.Td("11"), html.Td("Outlet fluid temperature"), html.Td("°C"), html.Td(get_t2_by_fluid()), html.Td()]),
                html.Tr([html.Td("12"), html.Td("Delta"), html.Td("°C"), html.Td(delta), html.Td()]),
                html.Tr([html.Td("13"), html.Td("Flow rate"), html.Td("L/min"), html.Td(get_flow_by_fluid()), html.Td()]),
                html.Tr([html.Td("14"), html.Td("Cooling capacity"), html.Td("kW"), html.Td(get_resa_by_fluid()), html.Td()]),
                html.Tr([html.Td("15"), html.Td("Tension"), html.Td("V"), html.Td(get_value_by_presa_elettrica()), html.Td()]),
                html.Tr([html.Td("16"), html.Td("Current"), html.Td("A"), html.Td(get_current_by_presa_elettrica()), html.Td()]),
                html.Tr([html.Td("17"), html.Td("Power"), html.Td("kW"), html.Td(get_power_by_presa_elettrica()), html.Td()]),
                html.Tr([html.Td(), html.Td(), html.Td(), html.Td(), html.Td()])
            ]
            
            results_content = html.Table(className='table', children=[
                html.Thead([
                    html.Tr([html.Th("Index"), html.Th("Parameter"), html.Th("Unit"), html.Th("Circuit 1"), html.Th("Circuit 2")])
                ]),
                html.Tbody(table_rows)
            ])
            
            return anagraph_content, results_content
    
    # Default empty tables
    anagraph_content = html.Table(className='table', children=[
                        html.Tbody([
                            #_____________________________________________
                            html.Tr([html.Td("Test ID: "), html.Td()]),
                            html.Tr([html.Td("Order: "), html.Td()]),
                            html.Tr([html.Td("Test number: "), html.Td()]),
                            html.Tr([html.Td("Date: ", style={'border-bottom': '3px solid grey'}), html.Td(style={'border-bottom': '3px solid grey'})]),
                            #______________________________________________
                            html.Tr([html.Td("Part number: "), html.Td()]),
                            html.Tr([html.Td("Revision: "), html.Td()]),
                            html.Tr([html.Td("Serial number: ", style={'border-bottom': '3px solid grey'}), html.Td(style={'border-bottom': '3px solid grey'})]),
                            #_______________________________________________
                            html.Tr([html.Td("Test bench: "), html.Td()]),
                            html.Tr([html.Td("Operator: "), html.Td()]),
                            html.Tr([html.Td("Hydraulic line: ", style={'border-bottom': '3px solid grey'}), html.Td(style={'border-bottom': '3px solid grey'})]),
                            #_______________________________________________
                            html.Tr([html.Td("Fluid: "), html.Td()]),
                            html.Tr([html.Td("Circuits: "), html.Td()]),  
                            html.Tr([html.Td("Refrigerant: "), html.Td()]),
                            html.Tr([html.Td("Configuration: ", style={'border-bottom': '3px solid grey'}), html.Td(style={'border-bottom': '3px solid grey'})]),
                            #_______________________________________________
                            html.Tr([html.Td("Duration: "), html.Td()]),
                            html.Tr([html.Td("Log file: "), html.Td()]),
                            html.Tr([html.Td("Last modified: ", style={'border-bottom': '3px solid grey'}), html.Td(style={'border-bottom': '3px solid grey'})]),
                            #_______________________________________________
                            html.Tr([html.Td("Status: "), html.Td()]),
                            html.Tr([html.Td("Approved by: "), html.Td()])

                        ])
                    ])
    results_content = html.Table(className='table', children=[
        html.Thead([
            html.Tr([html.Th("Index"), html.Th("Parameter"), html.Th("Unit"), html.Th("Circuit 1"), html.Th("Circuit 2")])
        ]),
        html.Tbody([
            html.Tr([html.Td("1"), html.Td("Condensation pressure"), html.Td("bar"), html.Td(), html.Td()]),
            html.Tr([html.Td("2"), html.Td("Condensation temperature"), html.Td("°C"), html.Td(), html.Td()]),
            html.Tr([html.Td("3"), html.Td("Compressor inlet temperature"), html.Td("°C"), html.Td(), html.Td()]),
            html.Tr([html.Td("4"), html.Td("Compressor outlet temperature"), html.Td("°C"), html.Td(), html.Td()]),
            html.Tr([html.Td("5"), html.Td("Evaporation pressure"), html.Td("bar"), html.Td(), html.Td()]),
            html.Tr([html.Td("6"), html.Td("Evaporation temperature"), html.Td("°C"), html.Td(), html.Td()]),
            html.Tr([html.Td("7"), html.Td("Subcooling"), html.Td("°C"), html.Td(), html.Td()]),
            html.Tr([html.Td("8"), html.Td("Superheating"), html.Td("°C"), html.Td(), html.Td()]),
            html.Tr([html.Td("9"), html.Td("Ambient temperature"), html.Td("°C"), html.Td(), html.Td()]),
            html.Tr([html.Td("10"), html.Td("Inlet fluid temperature"), html.Td("°C"), html.Td(), html.Td()]),
            html.Tr([html.Td("11"), html.Td("Outlet fluid temperature"), html.Td("°C"), html.Td(), html.Td()]),
            html.Tr([html.Td("12"), html.Td("Delta"), html.Td("°C"), html.Td(), html.Td()]),
            html.Tr([html.Td("13"), html.Td("Flow rate"), html.Td("L/min"), html.Td(), html.Td()]),
            html.Tr([html.Td("14"), html.Td("Cooling capacity"), html.Td("kW"), html.Td(), html.Td()]),
            html.Tr([html.Td("15"), html.Td("Tension"), html.Td("V"), html.Td(), html.Td()]),
            html.Tr([html.Td("16"), html.Td("Current"), html.Td("A"), html.Td(), html.Td()]),
            html.Tr([html.Td("17"), html.Td("Power"), html.Td("kW"), html.Td(), html.Td()]),
        ])
    ])
    
    return anagraph_content, results_content

# END CALL BACK TEST RESULTS______________________________________


#END MANAGEMENT DATA_________________________________________________________
# GANTT CHART________________________________________________________________

@app.callback(
    Output('gantt-chart', 'figure'),
    [Input('url', 'pathname'),
     Input('update-chart-button', 'n_clicks'),
     Input('reset-chart-button', 'n_clicks')],
    [State('management-table-draft', 'derived_virtual_data'),
     State('management-table-draft', 'derived_virtual_selected_rows')],
     allow_duplicate=True
)
def update_gantt_chart(pathname, update_clicks, reset_clicks, rows, selected_rows):
    if pathname == '/cell-report':
        # Determine which button was clicked
        ctx = dash.callback_context
        if not ctx.triggered:
            return {}
        button_id = ctx.triggered[0]['prop_id'].split('.')[0]

        if button_id == 'url':
            # Initial Gantt chart with the top 10 tasks
            initial_data = status_table.sort_values(by='DUE.DATE').head(10)
            return generate_gantt_chart(initial_data)
        elif button_id == 'update-chart-button':
            if selected_rows is None or len(selected_rows) == 0:
                return generate_gantt_chart(status_table.sort_values(by='DUE.DATE').head(10))
            selected_data = pd.DataFrame([rows[i] for i in selected_rows])
            return generate_gantt_chart(selected_data)
        elif button_id == 'reset-chart-button':
            # Reset to the initial Gantt chart with the top 10 tasks
            initial_data = status_table.sort_values(by='DUE.DATE').head(10)
            return generate_gantt_chart(initial_data)
    return {}
#END GANTT CHART________________________________________________________________
#UPDATE PLAN DATA TABLE_________________________________________________________
# Define the after_request function to add headers
@app.server.after_request
def add_header(response):
    response.cache_control.no_store = True
    return response

@app.callback(
    [Output('management-table-draft', 'data'),
     Output('management-table-draft', 'selected_rows')],
    [Input('submit-button', 'n_clicks'),
     Input('reset-button', 'n_clicks'),
     Input('update-database-button', 'n_clicks'),
     Input('select-all-button', 'n_clicks'),
     Input('deselect-all-button', 'n_clicks'),
     Input('url', 'pathname')],
    [State('date-picker-range', 'start_date'),
     State('date-picker-range', 'end_date'),
     State('selected_cell', 'value'),
     State('status_cell', 'value'),
     State('management-table-draft', 'data')]
)
def update_table_and_selection(submit_clicks, reset_clicks, update_db_clicks, select_all_clicks, deselect_all_clicks, pathname, start_date, end_date, selected_cells, selected_status, table_data):
    ctx = dash.callback_context

    # Reload the DataFrame from the updated database file
    test_data_new = pd.read_excel('Plan.xlsx', header=0, sheet_name='TEST')

    if not ctx.triggered:
        print("Page loaded or reloaded")
        return test_data_new.to_dict("records"), []

    button_id = ctx.triggered[0]['prop_id'].split('.')[0]
    print(f"Button clicked: {button_id}")

    if button_id == 'reset-button':
        print("Reset button clicked")
        return [], []

    if button_id == 'update-database-button':
        print("Update Database button clicked")
        
        # Update the database with the new values from the table
        updated_df = pd.DataFrame(table_data)
        
        # Ensure that the indices are aligned before comparing
        updated_df.set_index('TEST.ID', inplace=True)
        test_data_new.set_index('TEST.ID', inplace=True)
        
        # Debugging: Print only the modified cells before and after the update
        modified_cells_before = test_data_new.loc[test_data_new['TEST.STATUS'] != updated_df['TEST.STATUS']]
        modified_cells_after = updated_df.loc[test_data_new['TEST.STATUS'] != updated_df['TEST.STATUS']]
        
        print("Modified Cells Before Update:")
        print(modified_cells_before[['TEST.STATUS']])
        
        print("Modified Cells After Update:")
        print(modified_cells_after[['TEST.STATUS']])

        updated_df['DUE.DATE'] = pd.to_datetime(updated_df['DUE.DATE'], format='%d/%m/%Y', errors='coerce')
        updated_df['EXPECTED START'] = pd.to_datetime(updated_df['EXPECTED START'], format='%d/%m/%Y', errors='coerce')

        # Load the original database with all sheets
        original_df = pd.read_excel('Plan.xlsx', sheet_name=None)

        # Get the specific sheet to update
        sheet_name = 'TEST'
        original_sheet_df = original_df[sheet_name]

        # Ensure that the indices are aligned before merging
        original_sheet_df.set_index('TEST.ID', inplace=True)
        
        # Merge the updated data with the original data
        merged_df = original_sheet_df.combine_first(updated_df).reset_index()
        
        # Debugging: Print only the modified cells in the merged DataFrame
        modified_merged_cells = merged_df.loc[merged_df['TEST.STATUS'] != original_sheet_df['TEST.STATUS']]
        
        print("Modified Cells in Merged DataFrame:")
        print(modified_merged_cells[['TEST.ID', 'TEST.STATUS']])

        # Save the updated DataFrame back to the original Excel file without deleting other sheets
        try:
            with pd.ExcelWriter('Plan.xlsx', engine='openpyxl') as writer:
                for sheet in original_df.keys():
                    if sheet == sheet_name:
                        merged_df.to_excel(writer, sheet_name=sheet, index=False)
                    else:
                        original_df[sheet].to_excel(writer, sheet_name=sheet, index=False)
            print("Excel file has been updated successfully.")
        
        except Exception as e:
            print(f"Error updating Excel file: {e}")

        # Reload the DataFrame from the updated database file after updating
        test_data_new = pd.read_excel('Plan.xlsx', header=0, sheet_name='TEST')

        # Check the value of the 'TEST.STATUS' column for a specific row
        test_id = 'L000386'  # Example test ID
        status_value = test_data_new.loc[test_data_new.index == test_id, 'TEST.STATUS'].values[0]
        print(f"The value of TEST.STATUS for TEST.ID {test_id} is: {status_value}")

    if button_id == 'select-all-button':
        print("Select All button clicked")
        return test_data_new.to_dict("records"), list(range(len(test_data_new)))

    elif button_id == 'deselect-all-button':
        print("Deselect All button clicked")
        return test_data_new.to_dict("records"), []

    if button_id == 'submit-button':
        print("Submit button clicked")
        test_data_new['DUE.DATE'] = pd.to_datetime(test_data_new['DUE.DATE'], format='%d/%m/%Y', errors='coerce')
        test_data_new['EXPECTED START'] = pd.to_datetime(test_data_new['EXPECTED START'], format='%d/%m/%Y', errors='coerce')

        filtered_data = test_data_new.copy()
        print("Filtered Data before applying filters:")
        print(filtered_data)

        if start_date and end_date:
            start_date = pd.to_datetime(start_date)
            end_date = pd.to_datetime(end_date)
            filtered_data = filtered_data[(filtered_data['DUE.DATE'] >= start_date) & (filtered_data['DUE.DATE'] <= end_date)]
            print("Filtered Data after applying date filters:")
            print(filtered_data)

        if selected_cells:
            filtered_data = filtered_data[filtered_data['CELL'].isin(selected_cells)]
            print("Filtered Data after applying cell filters:")
            print(filtered_data)

        if selected_status and selected_status != 'ALL':
            if isinstance(selected_status, str):
                selected_status = [selected_status]
            filtered_data = filtered_data[filtered_data['TEST.STATUS'].isin(selected_status)]
            print("Filtered Data after applying status filters:")
            print(filtered_data)

        filtered_data = filtered_data.sort_values(by='DUE.DATE', ascending=False)
        filtered_data['DUE.DATE'] = filtered_data['DUE.DATE'].dt.strftime('%d/%m/%Y')
        filtered_data['EXPECTED START'] = filtered_data['EXPECTED START'].dt.strftime('%d/%m/%Y')

        return filtered_data.to_dict("records"), []

    return test_data_new.to_dict("records"), []
if __name__ == '__main__':
    webbrowser.open_new('http://127.0.0.1:8050/')
    app.run_server(debug=True)

Page loaded or reloaded
